In [1]:
import mosek.fusion as mf
from typing import List, Dict, Any
import pandas as pd
import numpy as np
import yfinance as yf
import altair as alt
alt.renderers.set_embed_options(actions=False, theme="dark")

RendererRegistry.enable('default')

In [12]:
def build_model(
    n_assets: int,
    asset_sectors: pd.DataFrame,
    constraints: List[Dict[str, Any]],
    risk_free: float = 0.0,
) -> mf.Model:
    """
    Function for building MOSEK model to solve the mean-variance optimization
    problem with diversification constraints.

    Parameters
    ----------
    n_assets : int
        Number of assets in the investment universe.
    asset_sectors : pd.DataFrame
        DataFrame containing assets' names (`"Asset"` column) and their
        corresponding sectors (`"Sector"` column).
    constraints : List[Dict[str, Any]]
        List of diversification constraints (dictionaries). The dictionaries
        must have the following keys:

        - `"Type"`: type of constraint. It can be `"All Assets"` (the constraint
        applies to all the assets), `"Sectors"` (the constraint applies only
        to assets from a particular sector) or `"Assets"` (the constraint
        applies to a particular asset).
        - `"Weight"`: limit value for the assets' weights.
        - `"Sign"`: domain of the constraint. It can be `">="` (greater than) or
        `"<="` (less than).
        - `"Position"`: indicates to which positions the constraint applies. It
        can be the name of a sector, the name of an asset or an empty string
        (`""`) if the constraint type is `"All Assets"`.
    risk_free: float
        Risk free rate.
        
    Returns
    -------
    model: mf.Model
        MOSEK model object.
    """
    # Creating the model
    model = mf.Model("sharpe_ratio")

    # Auxiliar vector variable y.
    y = model.variable("y", n_assets, mf.Domain.greaterThan(0.0))

    # Auxiliar scalar variable z.
    z = model.variable("z", 1, mf.Domain.greaterThan(0.0))

    # Variable for modeling the portfolio variance in the objective function
    s = model.variable("s", 1, mf.Domain.unbounded())

    # Parameter for cov matrix decomposition.
    G = model.parameter("G", [n_assets, n_assets])

    # Parameter for expected returns vector.
    mu = model.parameter("mu", n_assets)

    # Reformulation constraint
    model.constraint(
        "reformulation",
        mf.Expr.sub(mf.Expr.dot(mu, y), mf.Expr.mul(risk_free, z)),
        mf.Domain.equalsTo(1.0),
    )
    # Budget constraint (fully invested)
    model.constraint(
        "budget", mf.Expr.sub(mf.Expr.sum(y), z), mf.Domain.equalsTo(0)
    )

    # Iterate over the constraints list and add the constraints to the model.
    for c, constraint in enumerate(constraints):
        sign = (
            mf.Domain.greaterThan(0.0)
            if constraint["Sign"] == ">="
            else mf.Domain.lessThan(0.0)
        )

        if constraint["Type"] == "All Assets":
            A = np.identity(n_assets)

            model.constraint(
                f"c_{c}",
                mf.Expr.sub(
                    mf.Expr.mul(A, y),
                    mf.Expr.mul(
                        constraint["Weight"], mf.Var.vrepeat(z, n_assets)
                    ),
                ),
                sign,
            )

        elif constraint["Type"] == "Sectors":
            A = np.where(
                asset_sectors.loc[:, "Sector"] == constraint["Position"],
                1.0,
                0.0,
            )

            model.constraint(
                f"c_{c}",
                mf.Expr.sub(
                    mf.Expr.dot(A, y),
                    mf.Expr.mul(
                        constraint["Weight"], z
                    ),
                ),
                sign,
            )

        elif constraint["Type"] == "Assets":
            A = np.where(
                asset_sectors.loc[:, "Asset"] == constraint["Position"],
                1.0,
                0.0,
            )

            model.constraint(
                f"c_{c}",
                mf.Expr.sub(
                    mf.Expr.dot(A, y),
                    mf.Expr.mul(
                        constraint["Weight"], z
                    ),
                ),
                sign,
            )

    # Conic constraint for the portfolio variance
    model.constraint(
        "risk", mf.Expr.vstack(s, mf.Expr.mul(G, y)), mf.Domain.inQCone()
    )

    # Define objective function
    model.objective(
        "obj",
        mf.ObjectiveSense.Minimize,
        s,
    )

    return model

In [13]:
asset_sectors = pd.DataFrame(
    {
        "Asset": [
            "NVDA", 
            "AMD", 
            "INTC", 
            "BAC",
            "JPM",
            "C",
            "MSFT",
            "GOOG",
            "META",
            "BTC-USD",
            "ETH-USD",
        ],
        "Sector": [
            "Electronic Technology", 
            "Electronic Technology", 
            "Electronic Technology",
            "Finance",
            "Finance",
            "Finance",
            "Technology Services",
            "Technology Services",
            "Technology Services",
            "Crypto",
            "Crypto"
        ]
    }
)

assets_data = yf.download(asset_sectors.loc[:, "Asset"].to_list())

[*********************100%***********************]  11 of 11 completed


In [14]:
assets_returns = (
    assets_data.loc[:, "Adj Close"]
    .pct_change()
    .loc[:, asset_sectors.loc[:, "Asset"]]
)

sigma = assets_returns.cov()

mu = assets_returns.mean()

G = pd.DataFrame(
    np.linalg.cholesky(sigma), index=sigma.index, columns=sigma.columns
)


In [15]:
constraints = [
    {"Type": "All Assets", "Weight": 0.2, "Sign": "<=", "Position": ""},
    {
        "Type": "Sectors",
        "Weight": 0.3,
        "Sign": ">=",
        "Position": "Electronic Technology",
    },
    {
        "Type": "Sectors",
        "Weight": 0.4,
        "Sign": "<=",
        "Position": "Electronic Technology",
    },
    {
        "Type": "Assets",
        "Weight": 0.05,
        "Sign": "<=",
        "Position": "META",
    },
    {
        "Type": "Sectors",
        "Weight": 0.1,
        "Sign": "<=",
        "Position": "Crypto",
    },
]


In [16]:
n_assets = len(asset_sectors)

# Build constrained model
constrained_model = build_model(n_assets, asset_sectors, constraints)

# Set required parameters.

constrained_model.getParameter("G").setValue(
    G.to_numpy().T
)  # Remember to transpose G.

constrained_model.getParameter("mu").setValue(mu.to_numpy())

# Solve optimization problem.
constrained_model.solve()

# Get optimal weights from the Model object.
weights = pd.Series(
    constrained_model.getVariable("y").level()
    / constrained_model.getVariable("z").level()[0],
    index=asset_sectors.loc[:, "Asset"],
    name="Constrained",
).to_frame()

# Build unconstrained model
unconstrained_model = build_model(n_assets, asset_sectors, [])

# Set required parameters.

unconstrained_model.getParameter("G").setValue(
    G.to_numpy().T
)  # Remember to transpose G.

unconstrained_model.getParameter("mu").setValue(mu.to_numpy())

# Solve optimization problem.
unconstrained_model.solve()

# Get optimal weights from the Model object.

weights.loc[:, "Unconstrained"] = unconstrained_model.getVariable("y").level() / unconstrained_model.getVariable("z").level()[0]

weights.style.format("{:.2%}")

,Constrained,Unconstrained
Asset,,
NVDA,20.00%,12.10%
AMD,1.11%,0.00%
INTC,8.89%,0.00%
BAC,0.00%,0.00%
JPM,15.00%,1.66%
C,0.00%,0.00%
MSFT,20.00%,29.50%
GOOG,20.00%,22.54%
META,5.00%,7.19%


In [17]:
weights = weights.stack().reset_index().rename({"level_1": "Optimization Type", 0: "Weight"}, axis=1)

In [18]:
c = (
    alt.Chart(weights)
    .mark_bar()
    .encode(
        x=alt.X("Optimization Type:N", axis=None),
        y=alt.Y("Weight:Q", axis=alt.Axis(format=".2%", title="Weight")),
        color=alt.Color(
            "Optimization Type:N",
            legend=alt.Legend(orient="bottom", title=None),
            scale=alt.Scale(range=["#e60049", "#0bb4ff"])
        ),
        column=alt.Column("Asset:N", spacing=5),
    )
)

c


/Users/matiasmacazaga/python/portfolio_optimization_with_mosek/.venv/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [19]:
c.save("./altair_plots/weights_comparison_sharpe_ratio.html")

In [28]:
def get_portfolio_sharpe_ratio(
    weights: np.ndarray | pd.Series,
    mu: np.ndarray | pd.Series,
    sigma: np.ndarray | pd.DataFrame,
    ann_factor: int = 252,
):
    """
    Computes the Annualized Sharpe Ratio.

    Parameters
    ----------
    weights : np.ndarray | pd.Series
        Portfolio weights.
    mu : np.ndarray | pd.Series
        Expected return.
    sigma : np.ndarray | pd.DataFrame
        Covariance matrix.
    ann_factor : int, optional
        Annualization factor, by default 252.

    Returns
    -------
    float
        Sharpe Ratio.
    """
    std_dev = np.sqrt(
        np.dot(
            np.dot(
                weights,
                sigma,
            ),
            weights,
        )
    )

    expected_return = np.dot(
        weights,
        mu,
    )

    return (expected_return/std_dev)*np.sqrt(ann_factor)

In [30]:
sharpe_ratio_constrained = get_portfolio_sharpe_ratio(
    weights=weights.loc[
                weights.loc[:, "Optimization Type"] == "Constrained", "Weight"
            ],
    sigma=sigma,
    mu=mu,
)

sharpe_ratio_unconstrained = get_portfolio_sharpe_ratio(
    weights=weights.loc[
                weights.loc[:, "Optimization Type"] == "Unconstrained", "Weight"
            ],
    sigma=sigma,
    mu=mu,
)

print(f"Ann. Sharpe Ratio Constrained Optimization: {sharpe_ratio_constrained:.2f}")
print(f"Ann. Sharpe Ratio Unconstrained Optimization: {sharpe_ratio_unconstrained:.2f}")

Ann. Sharpe Ratio Constrained Optimization: 0.95
Ann. Sharpe Ratio Unconstrained Optimization: 1.10
